<a href="https://colab.research.google.com/github/the-cafehopper/Research/blob/main/Indonesia_E_Commerce_Energy_202510.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas numpy statsmodels pyjanitor openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 4.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
# Paths
household_fp = "dehs2020_household_v2.csv"
individual_fp = "dehs2020_individual_v2.csv"
out_fp = "dehs2020_masterdataset.csv"

In [ ]:
# Load
hh = pd.read_csv(household_fp, low_memory=False)
ind = pd.read_csv(individual_fp, low_memory=False)

In [ ]:
# Detect merge key (defaults to hhid if present)
hh_cols_lower = {c.lower(): c for c in hh.columns}
ind_cols_lower = {c.lower(): c for c in ind.columns}

if "hhid" in hh_cols_lower and "hhid" in ind_cols_lower:
  hh_key, ind_key = hh_cols_lower["hhid"], ind_cols_lower["hhid"]
else:
  common = set(hh_cols_lower).intersection(ind_cols_lower)
  # heuristic fallback
  candidates = [c for c in common if "hh" in c or "house" in c or c.endswith("id")]
  if not candidates:
    raise ValueError("Could not auto-detect a common household key. Set hh_key and ind_key manually.")
  pick = candidates[0]
  hh_key, ind_key = hh_cols_lower[pick], ind_cols_lower[pick]

In [ ]:
# Select + rename household vars
var_map_hh = {
    "m1_iii_1" : "province_code", # province identifier
    "m1_iii_5" : "urban_rural", # 1=Urban, 2=Rural
    "m9_ii_21_ax" : "monthly_income", # income proxy
    "m9_iii_67e" : "digital_payment_raw", # 1 yes, 3 no
    "m9_iii_68c" : "ecommerce_raw", # Likert 1..5 (1-2 positive)
    "m1_vi_35" : "lighting_source" # 1 PLN, 2 Non-PLN
}
present_hh = {k: v for k, v in var_map_hh.items() if k in hh.columns}
hh_sub = hh[[hh_key] + list(present_hh.keys())].rename(columns=present_hh)
hh_sub = hh_sub.rename(columns={hh_key: "hhid_key"})

In [ ]:
# Recode helpers
def yesno(series, yes=(1,), no=(3,)):
  return series.apply(lambda x: 1 if x in yes else (0 if x in no else np.nan))

# Digital payment (1=yes, 3=no)
if "digital_payment_raw" in hh_sub:
  hh_sub["digital_payment"] = yesno(hh_sub["digital_payment_raw"], yes=(1,), no=(3,))

# E-commerce (Likert 1..5 -> engaged=1 if 1 or 2; else 0
if "ecommerce_raw" in hh_sub:
  hh_sub["ecommerce_participation"] = hh_sub["ecommerce_raw"].apply(
      lambda x: 1 if x in [1,2] else (0 if x in [3,4,5] else np.nan)
  )

# Renewable proxy (Non-PLN -> 1; PLN -> 0)
if "lighting_source" in hh_sub:
  hh_sub["renewable_use"] = hh_sub["lighting_source"].apply(
      lambda x: 1 if x == 2 else (0 if x == 1 else np.nan)
  )

# Urban dummy
if "urban_rural" in hh_sub:
  hh_sub["urban_dummy"] = hh_sub["urban_rural"].apply(
      lambda x: 1 if x == 1 else (0 if x == 2 else np.nan)
  )

# Log income (avoid log(0))
if "monthly_income" in hh_sub:
  hh_sub["log_income"] = np.log(hh_sub["monthly_income"].replace({0: np.nan}))

# Composite digital participation
digital_cols = [c for c in ["digital_payment", "ecommerce_participation"] if c in hh_sub.columns]
if digital_cols:
  hh_sub["digital_participation"] = hh_sub[digital_cols].max(axis=1, skipna=True)

In [ ]:
# Education from individual file
if "m1_v_8" not in ind.columns:
    raise ValueError("Education variable 'm1_v_8' not found in individual data.")

# Rename merge key
ind_tmp = ind.rename(columns={ind_key: "hhid_key"})

# Head’s education (relation == 1)
rel_candidates = [c for c in ind_tmp.columns if c.lower() in {"m1_iv_2", "m1_ii_1", "relation", "rel_head"}]

if rel_candidates and rel_candidates[0] in ind_tmp.columns:
    rel_col = rel_candidates[0]
    # Filter for household head only (relation == 1)
    edu_hh = (
        ind_tmp.loc[ind_tmp[rel_col] == 1, ["hhid_key", "m1_v_8"]]
        .dropna()
        .groupby("hhid_key", as_index=False)["m1_v_8"]
        .first()
        .rename(columns={"m1_v_8": "education_level"})
    )
    print(f"Education aggregated using household head column: {rel_col}")
else:
    # Fallback: take MAX education within HH
    edu_hh = (
        ind_tmp.groupby("hhid_key", as_index=False)["m1_v_8"]
        .max()
        .rename(columns={"m1_v_8": "education_level"})
    )
    print("Education aggregated using MAX education within household (head not detected).")


Education aggregated using MAX education within household (head not detected).


In [ ]:
# Merge education into household data
df = hh_sub.merge(edu_hh, on="hhid_key", how="left")
print(merged[["hhid_key", "education_level"]].head())

   hhid_key  education_level
0      1103              7.0
1      1203              7.0
2      1105              7.0
3      1102              9.0
4      1205              7.0


In [ ]:
# Save the file for analysis
df.to_csv(out_fp, index=False)
print("Saved:", out_fp)
print("Columns:", df.columns.tolist())
print(df.head(3))

Saved: dehs2020_masterdataset.csv
Columns: ['hhid_key', 'province_code', 'urban_rural', 'monthly_income', 'digital_payment_raw', 'ecommerce_raw', 'lighting_source', 'digital_payment', 'ecommerce_participation', 'renewable_use', 'urban_dummy', 'log_income', 'digital_participation', 'education_level']
   hhid_key  province_code  urban_rural  monthly_income  digital_payment_raw  \
0      1103             11            2             1.0                  NaN   
1      1203             11            2             NaN                  NaN   
2      1105             11            2             1.0                  NaN   

   ecommerce_raw  lighting_source  digital_payment  ecommerce_participation  \
0            NaN                1              NaN                      NaN   
1            NaN                1              NaN                      NaN   
2            NaN                1              NaN                      NaN   

   renewable_use  urban_dummy  log_income  digital_participat

In [ ]:
# Quick diagnosis and descriptives
df = pd.read_csv("dehs2020_masterdataset.csv")

keep = [c for c in ["renewable_use","digital_payment","ecommerce_participation",
                    "digital_participation","log_income","urban_dummy",
                    "education_level","province_code"] if c in df.columns]
print("Available:", keep)

desc = df[keep].describe().T
desc["mean_pct"] = (desc["mean"]*100).round(2)
print(desc[["count","mean","mean_pct","std","min","max"]])

Available: ['renewable_use', 'digital_payment', 'ecommerce_participation', 'digital_participation', 'log_income', 'urban_dummy', 'education_level', 'province_code']
                          count       mean  mean_pct        std   min  \
renewable_use            3054.0   0.009168      0.92   0.095327   0.0   
digital_payment           191.0   0.115183     11.52   0.320082   0.0   
ecommerce_participation   191.0   0.685864     68.59   0.465391   0.0   
digital_participation     191.0   0.712042     71.20   0.454002   0.0   
log_income               1542.0   0.024100      2.41   0.221877   0.0   
urban_dummy              3063.0   0.666993     66.70   0.471366   0.0   
education_level          3047.0   8.442402    844.24   4.183718   1.0   
province_code            3063.0  58.303950   5830.40  24.337492  11.0   

                               max  
renewable_use             1.000000  
digital_payment           1.000000  
ecommerce_participation   1.000000  
digital_participation     1.0

In [ ]:
# Baseline models (LPM + penalized logit)
import statsmodels.formula.api as smf
import statsmodels.api as sm

d = pd.read_csv("dehs2020_masterdataset.csv")

# RHS builder
digital_rhs = [c for c in ["digital_payment","ecommerce_participation"] if c in d.columns]
if not digital_rhs and "digital_participation" in d.columns:
  digital_rhs = ["digital_participation"]

controls = [c for c in ["log_income","urban_dummy","education_level","province_code"] if c in d.columns]
rhs = " + ".join(digital_rhs + [c for c in controls if c!="province_code"] + (["C(province_code)"] if "province_code" in controls else []))

dm = d.dropna(subset=["renewable_use"] + digital_rhs + [c for c in controls if c!="province_code"] + (["province_code"] if "province_code" in controls else []))

print("N (analysis sample):", len(dm), "| renewable_use mean:", dm["renewable_use"].mean().round(4))

# LPM (robust SE)
lpm = smf.ols(f"renewable_use ~ {rhs}", data=dm).fit(cov_type="HC1")
print(lpm.summary())

# Penalized logit (L2) to handle rare outcome / separation
# Build design matrices from formula (evaluate once)
logit_model = smf.logit(f"renewable_use ~ {rhs}", data=dm)
y, X = logit_model.endog, sm.add_constant(logit_model.exog, has_constant='add')

# L2 penalty via fit_regularized (alpha is penalty strength; tune 0.1-5.0)
logit_pen = sm.Logit(y, X).fit_regularized(alpha=1.0, L1_wt=0.0, maxiter=200)
print("\nPenalized logit coefficients:\n", pd.Series(logit_pen.params, index=["const"]+logit_model.exog_names))

# Clustered SE for LPM by province
if "province_code" in dm.columns:
  lpm_cl = smf.ols(f"renewable_use ~ {rhs}", data=dm).fit(
      cov_type="cluster", cov_kwds={"groups": dm["province_code"]}
  )
  print("\nLPM with province-clustered SE:\n", lpm_cl.summary())

N (analysis sample): 191 | renewable_use mean: 0.0
                            OLS Regression Results                            
Dep. Variable:          renewable_use   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 13 Oct 2025   Prob (F-statistic):                nan
Time:                        07:12:00   Log-Likelihood:                    inf
No. Observations:                 191   AIC:                              -inf
Df Residuals:                     160   BIC:                              -inf
Df Model:                          30                                         
Covariance Type:                  HC1                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------

/usr/local/lib/python3.12/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 30, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1923: RuntimeWarning: invalid value encountered in divide
  F /= J
/usr/local/lib/python3.12/dist-packages/statsmodels/regression/linear_model.py:955: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/usr/local/lib/python3.12/dist-packages/statsmodels/stats/stattools.py:50: RuntimeWarning: invalid value encountered in scalar divide
  dw = np.sum(diff_resids**2, axis=axis) / np.sum(resids**2, axis=axis)
/usr/local/lib/python3.12/dis

In [ ]:
# heterogeneity (income or rural)
# Interactions
if set(["log_income","digital_participation"]).issubset(dm.columns):
  lpm_int = smf.ols("renewable_use ~ digital_participation*log_income + urban_dummy + education_level + C(province_code)", data=dm).fit(cov_type="HC1")
  print(lpm_int.summary())

# Rural only
if "urban_dummy" in dm.columns:
  dm_rural = dm[dm["urban_dummy"]==0].copy()
  if len(dm_rural) >50:
    lpm_rural = smf.ols(f"renewable_use ~ {rhs}", data=dm_rural).fit(cov_type="HC1")
    print("\nRural only LPM:\n", lpm_rural.summary())

                            OLS Regression Results                            
Dep. Variable:          renewable_use   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 13 Oct 2025   Prob (F-statistic):                nan
Time:                        07:18:27   Log-Likelihood:                    inf
No. Observations:                 191   AIC:                              -inf
Df Residuals:                     160   BIC:                              -inf
Df Model:                          30                                         
Covariance Type:                  HC1                                         
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.12/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 30, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1923: RuntimeWarning: invalid value encountered in divide
  F /= J
/usr/local/lib/python3.12/dist-packages/statsmodels/regression/linear_model.py:955: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/usr/local/lib/python3.12/dist-packages/statsmodels/stats/stattools.py:50: RuntimeWarning: invalid value encountered in scalar divide
  dw = np.sum(diff_resids**2, axis=axis) / np.sum(resids**2, axis=axis)


In [ ]:
# Export clean tables (for markdown)
def simple_table(result, title="Table", roundto=3):
  coefs = result.params.round(roundto)
  ses = result.bse.round(roundto) if hasattr(result, "bse") else None
  rows = []
  for name in coefs.index:
    if name == "intercept" or name == "const":
      label = "Constant"
    else:
      label = name
    if ses is not None and name in ses.index:
      rows.append(f"| {label} | {coefs[name]} | |")
  header = f"### {title}\n\n| Variable | Coef. | SE |\n|---|---:|---:|\n"
  return header + "\n".join(rows)

print(simple_table(lpm, "Baseline LPM (HC1)"))
if "lpm_cl" in locals():
    print(simple_table(lpm_cl, "Baseline LPM (Province-clustered)"))

### Baseline LPM (HC1)

| Variable | Coef. | SE |
|---|---:|---:|
| Intercept | 0.0 | |
| C(province_code)[T.12] | 0.0 | |
| C(province_code)[T.13] | 0.0 | |
| C(province_code)[T.15] | 0.0 | |
| C(province_code)[T.16] | 0.0 | |
| C(province_code)[T.17] | 0.0 | |
| C(province_code)[T.18] | 0.0 | |
| C(province_code)[T.19] | 0.0 | |
| C(province_code)[T.31] | 0.0 | |
| C(province_code)[T.32] | 0.0 | |
| C(province_code)[T.33] | 0.0 | |
| C(province_code)[T.35] | 0.0 | |
| C(province_code)[T.36] | 0.0 | |
| C(province_code)[T.52] | 0.0 | |
| C(province_code)[T.53] | 0.0 | |
| C(province_code)[T.61] | 0.0 | |
| C(province_code)[T.62] | 0.0 | |
| C(province_code)[T.63] | 0.0 | |
| C(province_code)[T.64] | 0.0 | |
| C(province_code)[T.71] | 0.0 | |
| C(province_code)[T.72] | 0.0 | |
| C(province_code)[T.73] | 0.0 | |
| C(province_code)[T.74] | 0.0 | |
| C(province_code)[T.81] | 0.0 | |
| C(province_code)[T.91] | 0.0 | |
| C(province_code)[T.94] | 0.0 | |
| digital_payment | 0.0 | |
| ecommer

In [ ]:
# Regression troubleshooting
# Dependent variable (renewable_use) has no variation and the regression couldnt estimate anything meaningful
# Check if renewable_use actually varies
dm["renewable_use"].value_counts(dropna=False)

,count
renewable_use,
0.0,191


In [ ]:
# Every observation is non-renewable ir. renewable_use = 0 for all households
# Investigate raw variable
print(hh["m1_vi_35"].value_counts(dropna=False))

m1_vi_35
1    3026
2      28
3       9
Name: count, dtype: int64


In [ ]:
# Seems like when the dataset was merged and cleanred, the regression sample (dm) dropped all the 2s and 3s so the regression coefficients are all 0
# Confrim which households were dropped
print(hh_sub["renewable_use"].value_counts(dropna=False))
print(d["renewable_use"].value_counts(dropna=False))
print(dm["renewable_use"].value_counts(dropna=False))

renewable_use
0.0    3026
1.0      28
NaN       9
Name: count, dtype: int64
renewable_use
0.0    3026
1.0      28
NaN       9
Name: count, dtype: int64
renewable_use
0.0    191
Name: count, dtype: int64


In [ ]:
# Adjust the dependent variable definition
# Recode the renewable variable more inclusively: 1=PLN(on-grid), 2=Non-PLN(off-grid), 3=PLN+Non-PLN(hybrid)
hh_sub["renewable_use"] = hh_sub["lighting_source"].map({1: 0, 2: 1, 3: 1})

hh_sub["renewable_use"].value_counts(dropna=False)

,count
renewable_use,
0,3026
1,37


In [ ]:
# Re-merge and re-run regression
df = hh_sub.merge(edu_hh, on="hhid_key", how="left")
df.to_csv("dehs2020_masterdataset.csv", index=False)

# Same LPM model code as before
lpm = smf.ols(f"renewable_use ~ {rhs}", data=df).fit(cov_type="HC1")
print(lpm.summary())

                            OLS Regression Results                            
Dep. Variable:          renewable_use   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 13 Oct 2025   Prob (F-statistic):                nan
Time:                        08:46:24   Log-Likelihood:                    inf
No. Observations:                 191   AIC:                              -inf
Df Residuals:                     160   BIC:                              -inf
Df Model:                          30                                         
Covariance Type:                  HC1                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

/usr/local/lib/python3.12/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 31, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1923: RuntimeWarning: invalid value encountered in divide
  F /= J
/usr/local/lib/python3.12/dist-packages/statsmodels/regression/linear_model.py:955: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/usr/local/lib/python3.12/dist-packages/statsmodels/stats/stattools.py:50: RuntimeWarning: invalid value encountered in scalar divide
  dw = np.sum(diff_resids**2, axis=axis) / np.sum(resids**2, axis=axis)


In [ ]:
# The regression is still collapsing even though m1_vi_35 has been recoded to include 2 and 3 as renewable; the regression sample (dm, 191 obs) still contains only 0s
# Quick diagnostic
# Check overall dataset before filtering
print(df["renewable_use"].value_counts(dropna=False))

# Check modeling sample after dropna
print(dm["renewable_use"].value_counts(dropna=False))

# See what rows got dropped
print(len(df), "total rows in df")
print(len(dm), "rows used in regression")

renewable_use
0    3026
1      37
Name: count, dtype: int64
renewable_use
0.0    191
Name: count, dtype: int64
3063 total rows in df
191 rows used in regression


In [ ]:
# Relax the filtering so it only require renewable_use to be nonmissing
# Keep all renewable households
# Rebuild the model dataset, keeping renewable_use nonmissing
needed = ["renewable_use"] + digital_rhs + controls
if "province_code" in d.columns and "province_code" not in needed:
  needed.append("province_code")

dm = d[needed].copy()
dm = dm.dropna(subset=["renewable_use"])

print("After relaxed filtering:")
print(dm["renewable_use"].value_counts(dropna=False))

After relaxed filtering:
renewable_use
0.0    3026
1.0      28
Name: count, dtype: int64


In [ ]:
# Recreate digital_participation
# Build from available digital flags
cand = [c for c in ["digital_payment","ecommerce_participation"] if c in dm.columns]
if "digital_participation" not in dm.columns and cand:
  dm["digital_participation"] = dm[cand].max(axis=1, skipna=True)

# If neither digital var exists, stop and show whats missing
if not [c for c in ["digital_participation","digital_payment","ecommerce_participation"] if c in dm.columns]:
  raise ValueError("No digital variables present. Ensure digital_payment / ecommerce_participation were created earlier.")

# Make sure key variables are numeric
to_num = [c for c in ["renewable_use","digital_participation","digital_payment",
                      "ecommerce_participation","log_income","urban_dummy",
                      "education_level"] if c in dm.columns]
for c in to_num:
  dm[c] = pd.to_numeric(dm[c], errors="coerce")

# Build a simple RHS that adapts to whats present
digital_rhs = [c for c in ["digital_participation","digital_payment","ecommerce_participation"] if c in dm.columns]
controls = [c for c in ["log_income","urban_dummy","education_level"] if c in dm.columns]
rhs = " + ".join(digital_rhs + controls)

# Keep rows with DV and at least the RHS being used
d_run = dm.dropna(subset=["renewable_use"] + digital_rhs + controls)
print("N used:", len(d_run))
print("Renewable use distribution:\n", d_run["renewable_use"].value_counts(dropna=False))

N used: 191
Renewable use distribution:
 renewable_use
0.0    191
Name: count, dtype: int64


In [ ]:
# Run a simpler model: no province FE yet
lpm_simple = smf.ols(f"renewable_use ~ {rhs}", data=d_run).fit(cov_type="HC1")
print(lpm_simple.summary())

                            OLS Regression Results                            
Dep. Variable:          renewable_use   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 13 Oct 2025   Prob (F-statistic):                nan
Time:                        09:23:46   Log-Likelihood:                    inf
No. Observations:                 191   AIC:                              -inf
Df Residuals:                     184   BIC:                              -inf
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

/usr/local/lib/python3.12/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1923: RuntimeWarning: invalid value encountered in divide
  F /= J
/usr/local/lib/python3.12/dist-packages/statsmodels/regression/linear_model.py:955: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/usr/local/lib/python3.12/dist-packages/statsmodels/stats/stattools.py:50: RuntimeWarning: invalid value encountered in scalar divide
  dw = np.sum(diff_resids**2, axis=axis) / np.sum(resids**2, axis=axis)


In [ ]:
# The regression result still shows all 0s; may be because an older filtered subset (dm) from before relaxing the filtering is being used
# Recreate a flesh dataset for modeling
# Rebuild a clean version from the full df (2026 + 28 households)
dm = df.copy()
cols = ["renewable_use", "digital_participation","log_income","urban_dummy","education_level"]
dm = dm[cols].dropna(subset=["renewable_use"])
print("Fresh sample size:", len(dm))

Fresh sample size: 3063


In [ ]:
# Rerun the same regression model
# Run the baseline regression
lpm_simple = smf.ols(
    "renewable_use ~ digital_participation + log_income + urban_dummy + education_level",
    data=dm
).fit(cov_type="HC1")

print(lpm_simple.summary())

                            OLS Regression Results                            
Dep. Variable:          renewable_use   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 13 Oct 2025   Prob (F-statistic):                nan
Time:                        10:22:12   Log-Likelihood:                    inf
No. Observations:                 191   AIC:                              -inf
Df Residuals:                     186   BIC:                              -inf
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                     

/usr/local/lib/python3.12/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1923: RuntimeWarning: invalid value encountered in divide
  F /= J
/usr/local/lib/python3.12/dist-packages/statsmodels/regression/linear_model.py:955: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/usr/local/lib/python3.12/dist-packages/statsmodels/stats/stattools.py:50: RuntimeWarning: invalid value encountered in scalar divide
  dw = np.sum(diff_resids**2, axis=axis) / np.sum(resids**2, axis=axis)


In [ ]:
# Further issues inspection and fix
# See which variables are causing the drop
# Runt a diagnostic
missing_summary = dm.isna().sum()
print(missing_summary)

renewable_use               0
digital_participation    2872
log_income               1521
urban_dummy                 0
education_level            16
dtype: int64


In [ ]:
# Check whether nearly all the rows have NaN in at least one of those columns
dm_clean = dm.dropna(subset=["renewable_use","digital_participation","log_income","urban_dummy","education_level"])
print("Rows remaining after full non-missing filter:", len(dm_clean))

Rows remaining after full non-missing filter: 191


In [ ]:
# Inspect why the NaNs exist
for col in ["digital_participation","log_income","urban_dummy","education_level"]:
  print(col, dm[col].unique()[:10])

digital_participation [nan  0.  1.]
log_income [0.                nan 2.07944154 1.94591015]
urban_dummy [0 1]
education_level [ 7.  9. 13.  2.  4. 11. 12.  8.  1. 14.]


In [ ]:
# Quick patch: recode missing data for now
dm_test = dm.fillna({
    "digital_participation": 0,
    "digital_payment": 0,
    "ecommerce_participation": 0,
    "log_income": dm["log_income"].median(),
    "urban_dummy": dm["urban_dummy"].mode()[0],
    "education_level": dm["education_level"].median()
})

In [ ]:
# Rerun the model
lpm_simple = smf.ols(
    "renewable_use ~ digital_participation + log_income + urban_dummy + education_level",
    data=dm_test
).fit(cov_type="HC1")

print(lpm_simple.summary())

                            OLS Regression Results                            
Dep. Variable:          renewable_use   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     8.663
Date:                Mon, 13 Oct 2025   Prob (F-statistic):           5.97e-07
Time:                        10:35:38   Log-Likelihood:                 2460.7
No. Observations:                3063   AIC:                            -4911.
Df Residuals:                    3058   BIC:                            -4881.
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.03